In [ ]:
# !pip install kiwipiepy

In [ ]:
!pip install konlpy

In [ ]:
# !pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset

from sklearn.utils import resample

# import warnings
# %config InlineBackend.figure_format = 'retina'
# warnings.filterwarnings("ignore")
# %matplotlib inline

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/멀티캠퍼스 자료/Machine Learning/data/"

SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# 데이터 선별

- 전체 데이터

In [ ]:
yogiyo =pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0908.csv") #37000개
posneg_train = pd.read_csv(f"{DATA_PATH}posneg_train_37000.csv") #37000개

gpt_neg_data = pd.read_csv(f"{DATA_PATH}gpt_neg_0_8000.csv") #10000개

In [ ]:
train= yogiyo[['고객리뷰','사장댓글','별점']]
train.columns = ['review', 'reply','rating']

tmp = posneg_train[['sentiment','score']]
train = pd.concat([train,tmp],axis=1)

train

,review,reply,rating,sentiment,score
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다💖 \n\n보람차게 일할 수 있게 해 주...,5,1,0.566494
1,기름 있는 부분이 생각보다 많아요! 참고하세요:),소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,5,0,0.723872
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해드리고 싶습니다!😄 \n\n맛있는 메뉴를 넉넉하게 먹을 수 ...,5,1,0.608408
3,"..,........",저희 매장과 함께 편안한 시간 되셨을까요? \n\n1. 고객을 생각하는 마음 \n2...,5,0,0.499478
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요!😁\n\n고객님의 행복...,5,1,0.381793
...,...,...,...,...,...
37135,맛있어요 ㅎ저번에주문하고 맛있어서또주문핰건데\n늦은시간에시켜서그렄지 국물이 엄청 걸...,고객님 안녕하세요 짬뽕관입니다~~^-^\n소중한 리뷰 너무 감사합니다!!🙂\n재주문...,5,1,0.424215
37136,맛있어요 또시킬게요,고객님 안녕하요 짬뽕관입니다!!^_^\n소중한 리뷰 남겨주셔 서 정말 달감사합니다!...,5,1,0.720691
37137,정말 맛있게 잘먹었어요!!자주 시켜 먹을께요!,고객님 안녕하세요 짬뽕관입니다~~^_^\n맛있게 드셨다는 말씀에 힘이 나는거 같습니...,5,1,0.833007
37138,와우.. 여기뭐죠 주변에서 맛있다고 먹어보라해서 시켰는데 육수가 일단 시원칼칼하니 ...,고객님 안녕하세요 짬뽕관입니다!!^_^\n맛있게 드셔주시고 정성스러운 사진 리뷰까지...,5,1,0.900031


- 오버샘플링 (부정데이터 GPT)

In [ ]:
len((train[train['sentiment'] == 0])) # 실 부정데이터 : 1824

1824

In [ ]:
gpt = gpt_neg_data[['neg_review','neg_reply','rating','sentiment','score']] # 오버샘플 부정데이터 : 6907
gpt.columns = ['review','reply','rating','sentiment','score']
len(gpt)

6907

In [ ]:
gpt['sentiment'] = gpt['sentiment'].replace({'부정': 0, '긍정': 1})

<ipython-input-84-05e0f090f57f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gpt['sentiment'] = gpt['sentiment'].replace({'부정': 0, '긍정': 1})


In [ ]:
train = pd.concat([train,gpt],axis=0)
train

,review,reply,rating,sentiment,score
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다💖 \n\n보람차게 일할 수 있게 해 주...,5,1,0.566494
1,기름 있는 부분이 생각보다 많아요! 참고하세요:),소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,5,0,0.723872
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해드리고 싶습니다!😄 \n\n맛있는 메뉴를 넉넉하게 먹을 수 ...,5,1,0.608408
3,"..,........",저희 매장과 함께 편안한 시간 되셨을까요? \n\n1. 고객을 생각하는 마음 \n2...,5,0,0.499478
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요!😁\n\n고객님의 행복...,5,1,0.381793
...,...,...,...,...,...
6902,너무 매워서 못먹을 정도네요. 좀 덜 매운 옵션도 있었으면 좋겠어요,매움의 정도를 더욱 조절하여 다양한 선택지를 드릴 수 있도록 노력하겠습니다. 소중...,5,0,0.825807
6903,음식이 다 덜 익어서 먹을 수 없어요. 정말 실망했습니다,음식의 익음 정도에 대해서 실망하셨던 것 같네요. 보다 완벽한 음식을 제공하기 위...,5,0,0.935712
6904,배달 음식이라 그런지 음식의 특유한 풍미가 덜해요. 식당에서 먹는 거랑은 다른 것...,배달 음식의 특성상 환경 변화로 인해 맛에 차이가 있을 수 있습니다. 보다 향긋한...,5,0,0.906386
6905,음식이 너무 기름진 느낌이 있어서 좀 그랬어요. 좀 더 가볍게 해주셨으면 좋겠습니다,기름 느낌이 덜하도록 요리하여 보다 가벼운 맛을 드릴 수 있도록 노력하겠습니다. ...,5,0,0.879808


In [ ]:
train['review'] = train['review'].astype(str)
train['reply'] = train['reply'].astype(str)
train

,review,reply,rating,sentiment,score
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다💖 \n\n보람차게 일할 수 있게 해 주...,5,1,0.566494
1,기름 있는 부분이 생각보다 많아요! 참고하세요:),소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,5,0,0.723872
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해드리고 싶습니다!😄 \n\n맛있는 메뉴를 넉넉하게 먹을 수 ...,5,1,0.608408
3,"..,........",저희 매장과 함께 편안한 시간 되셨을까요? \n\n1. 고객을 생각하는 마음 \n2...,5,0,0.499478
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요!😁\n\n고객님의 행복...,5,1,0.381793
...,...,...,...,...,...
6902,너무 매워서 못먹을 정도네요. 좀 덜 매운 옵션도 있었으면 좋겠어요,매움의 정도를 더욱 조절하여 다양한 선택지를 드릴 수 있도록 노력하겠습니다. 소중...,5,0,0.825807
6903,음식이 다 덜 익어서 먹을 수 없어요. 정말 실망했습니다,음식의 익음 정도에 대해서 실망하셨던 것 같네요. 보다 완벽한 음식을 제공하기 위...,5,0,0.935712
6904,배달 음식이라 그런지 음식의 특유한 풍미가 덜해요. 식당에서 먹는 거랑은 다른 것...,배달 음식의 특성상 환경 변화로 인해 맛에 차이가 있을 수 있습니다. 보다 향긋한...,5,0,0.906386
6905,음식이 너무 기름진 느낌이 있어서 좀 그랬어요. 좀 더 가볍게 해주셨으면 좋겠습니다,기름 느낌이 덜하도록 요리하여 보다 가벼운 맛을 드릴 수 있도록 노력하겠습니다. ...,5,0,0.879808


## 정규표현식
- 앞 뒤 콤마, 공백 / 영어 / 중복 문장기호 등 제거

In [ ]:

review_list = []
for review in train['review']:
    if isinstance(review, str):

        pat = re.compile("[^가-힣0-9.,~?!]")
        result = pat.sub(" ", review)

        result = re.sub(r',\s*$', '', result)
        result = re.sub(r'^\s*,', '', result)
        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        review_list.append(result)
    else:
        review_list.append("")

train['review'] = review_list
train.head()


,review,reply,rating,sentiment,score
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다💖 \n\n보람차게 일할 수 있게 해 주...,5,1,0.566494
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,5,0,0.723872
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해드리고 싶습니다!😄 \n\n맛있는 메뉴를 넉넉하게 먹을 수 ...,5,1,0.608408
3,".,.",저희 매장과 함께 편안한 시간 되셨을까요? \n\n1. 고객을 생각하는 마음 \n2...,5,0,0.499478
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요!😁\n\n고객님의 행복...,5,1,0.381793


In [ ]:
reply_list = []
for reply in train['reply']:
    if isinstance(reply, str):

        pat = re.compile("[^가-힣0-9.,~?!]")
        result = pat.sub(" ", reply)

        result = re.sub(r',\s*$', '', result)
        result = re.sub(r'^\s*,', '', result)
        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        reply_list.append(result)
    else:
        reply_list.append("")

train['reply'] = reply_list
train.head()


,review,reply,rating,sentiment,score
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,1,0.566494
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,5,0,0.723872
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해드리고 싶습니다! 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,5,1,0.608408
3,".,.",저희 매장과 함께 편안한 시간 되셨을까요? 1. 고객을 생각하는 마음 2. 더 나은...,5,0,0.499478
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요! 고객님의 행복한 시간...,5,1,0.381793


## 짧은 리뷰 처리

In [ ]:
train[(train['review'].str.len() <= 3)] # 길이가 n자 미만

,review,reply,rating,sentiment,score
3,".,.",저희 매장과 함께 편안한 시간 되셨을까요? 1. 고객을 생각하는 마음 2. 더 나은...,5,0,0.499478
147,,안녕하세요 1988응답하라추억의옛날도시락 입니다 든든한 한 끼 되셨을까요? 가족에게...,4,1,0.387972
549,,안녕하세요 입에는 맞으셨나요? 주문 주셔서 감사합니다,5,1,0.387972
551,. .,떡볶이집에서 사이드만 잔뜩 시키고 1점?,1,0,0.477857
741,굿,안녕하세요! 당신은 지금 치킨이 땡긴다 헬리오시티점 입니다 맛있게 드셔주셔서 감사합...,5,1,0.614493
...,...,...,...,...,...
36445,짱~,리뷰도짱짱 감사합니다 나중에 생각마실때 또 찾아쥬세용~,5,1,0.735108
36742,굿 ~,고객님의 최고의 만족을 위해 노력하는 누보닭갈비 입니다! 고객님의 한끼 식사가 든든...,5,1,0.615289
36787,,안녕하세요~! 고객님의 한끼식사를 위해 항상 노력하는 누보닭갈비 입니다! 전 직원이...,4,1,0.387972
36790,,안녕하세요~! 고객님의 한끼식사를 위해 항상 노력하는 누보닭갈비 입니다! 전 직원이...,5,1,0.387972


In [ ]:
train[(train['reply'].str.len() <= 16)] # 길이가 n자 미만

,review,reply,rating,sentiment,score
417,맛있어요 배달 빠릅니당,번개배달이죠,5,1,0.585138
421,맛있게 잘 먹었어요. 잼이 화룡정점!,잼 맛있으세요? 감사합니다,5,1,0.452707
425,와플이 맛있네요 커피도 맛있어여,저도 자주 먹어요.,5,1,0.567609
427,포장 깔끔하고 맛있어요 ~,감사합니다,4,1,0.561284
432,한번 먹고 맛 들려서 계속 시켜요,감사합니다,3,1,0.607706
...,...,...,...,...,...
35496,맛잇어용~,맛있게 드셧다니 감사합니다,5,1,0.467936
35651,,음. 뭔가 아쉬우셨군요,3,1,0.387972
35944,맛있게 잘 먹었습니다,항상최선을 다하겠습니다,5,1,0.538841
36476,친절하시고 맛있구 배달도 빠르고 가성비도 좋아용! 담에는 다른 맛으로 또 시켜먹고 ...,감사합니다 맛있게 드세요 !,5,1,0.696545


In [ ]:
mask = (train['review'].str.len() <= 3) | (train['reply'].str.len() <= 16)
train = train[~mask] # 37140 -> 35593
train

,review,reply,rating,sentiment,score
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,1,0.566494
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,5,0,0.723872
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해드리고 싶습니다! 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,5,1,0.608408
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요! 고객님의 행복한 시간...,5,1,0.381793
5,맛나게 잘 묵었습니다,소중한 순간을 저희 브랜드와 함께 해주셔서 영광입니다. 다음에도 고객님의 즐거운 순...,5,1,0.549870
...,...,...,...,...,...
6902,너무 매워서 못먹을 정도네요. 좀 덜 매운 옵션도 있었으면 좋겠어요,매움의 정도를 더욱 조절하여 다양한 선택지를 드릴 수 있도록 노력하겠습니다. 소중...,5,0,0.825807
6903,음식이 다 덜 익어서 먹을 수 없어요. 정말 실망했습니다,음식의 익음 정도에 대해서 실망하셨던 것 같네요. 보다 완벽한 음식을 제공하기 위...,5,0,0.935712
6904,배달 음식이라 그런지 음식의 특유한 풍미가 덜해요. 식당에서 먹는 거랑은 다른 것...,배달 음식의 특성상 환경 변화로 인해 맛에 차이가 있을 수 있습니다. 보다 향긋한...,5,0,0.906386
6905,음식이 너무 기름진 느낌이 있어서 좀 그랬어요. 좀 더 가볍게 해주셨으면 좋겠습니다,기름 느낌이 덜하도록 요리하여 보다 가벼운 맛을 드릴 수 있도록 노력하겠습니다. ...,5,0,0.879808


## 최종 train (긍부정 1:1)

In [ ]:
train['sentiment'].value_counts().sort_index() # 긍부정 분포 확인

0     8680
1    34219
Name: sentiment, dtype: int64

In [ ]:
train_neg = train[train['sentiment'] == 0]

# 긍정 데이터 중 중복된 답변제거
train_pos = train[train['sentiment'] == 1].drop_duplicates(subset='reply', keep=False)
train['sentiment'].value_counts().sort_index()

0     8680
1    34219
Name: sentiment, dtype: int64

In [ ]:
train_neg = train_neg.sample(frac=1, random_state=42).reset_index(drop=True)
train_neg

,review,reply,rating,sentiment,score
0,음식이 신선하지 않았고 맛도 약했어요.,더 나은 제품을 위해 피드백을 감사히 받아들이겠습니다.,5,0,0.918361
1,비오는날인데 배달도 늦고 너무 맛없이 잘 먹엇습니다!,비오는 날에도 배달이 늦어서 정말 죄송합니다. 맛도 많이 부족하셨다니 안타까워요....,5,0,0.850407
2,주문한 음식이 식은 상태로 도착했어요.,고객님께 불편을 드린 점 정말 죄송합니다. 조속한 조치를 취하도록 하겠습니다.,5,0,0.904237
3,여기 음식이 정말 별로였어요. 맛이 없어서 다 못 먹었어요. 다시는 시키지 않을 ...,맛이 없으셨군요. 정말 죄송합니다. 다음에는 더 맛있는 음식을 제공할 수 있도록 ...,5,0,0.935421
4,맛이 아쉬웠어요. 좀 더 개선되었으면 좋겠어요.,음식의 양이 충분하지 않았다니 정말 죄송합니다. 더 많은 양의 음식을 제공할 수 있...,5,0,0.884297
...,...,...,...,...,...
8675,치킨은 맛있는 편이었지만 떡볶이가 딱딱해서 별로였어요. 다른 식당에서 시킬 걸 그...,죄송합니다. 떡볶이가 딱딱해서 신선하지 않았던 것 같네요. 다음 번에는 더 맛있는...,5,0,0.888545
8676,공기밥이 썩어서 먹을 수가 없었어요. 너무 아쉬워요,안타깝게도 공기밥이 썩어서 드실 수 없었다는 것에 대해 진심으로 사과드립니다. 더...,5,0,0.897410
8677,맛을 기대하고 시켰는데 실망했어요. 별로 맛이 없었어요,맛을 기대하고 시켰는데 실망했어요. 별로 맛이 없었어요,5,0,0.923196
8678,너무 식어서 와서 약간 아쉬웠어요 버거는 맛있는데 감튀는 아쉽네요 양도 적구요,배달은 최대한 시간 맞춰준비해도 어쩔수없이 식게 되네요 더욱 열심히 준비하겠습니다!,3,0,0.708976


In [ ]:
train_pos = train_pos.sample(frac=1, random_state=42).reset_index(drop=True)[:len(train_neg)]
train_pos

,review,reply,rating,sentiment,score
0,맛있게 잘 먹었습니다,"안녕하세요 소중한 고객님! 맛있게 드셨다니, 뿌듯합니다 언제나 기대 이상의 음식으로...",5,1,0.538841
1,맛잇게잘먹었습니다 다음엔 로제먹어봐야겟어요,저희 이배달을 믿고 주문해 주셔서 감사합니다 고객님 입맛에 맛있었다니 너무 다행입니...,5,1,0.542652
2,맛있게 잘먹었습니다 감사합니다,"저희 메뉴가 만족감을 드린 거 같아 다행이에요! 맛 뿐만 아니라 양도, 서비스도 모...",5,1,0.690875
3,인생 뼈해장국 너무 맛있고 반찬들다 진짜 맛있어요,"감사합니다,인생뼈 해장국 넘 좋네요 오늘하루 행복하게 보내세요",5,1,0.884303
4,맛있어서 두번째 주문입니다,정말정말 감사합니다 늘 변함없는 맛있는 맛으로 찾아 뵙겠습니다!,5,1,0.724946
...,...,...,...,...,...
8675,넘친절하시고마시써요,새해 복 많이 받으시고 건강챙기시구요! 항상 고객님에게 맛있는치킨 기억에 남을치킨을...,5,1,0.791029
8676,마라탕은 여기가 선택 메뉴가 젤 많아서 최고에요 ! 언제나 잘 먹고 있습니다 ~,안녕하세요! 주문해주셔서 감사합니다 소중한 별점과 리뷰 넘 감사드립니당 언제나 맛있...,5,1,0.671336
8677,잘 먹었습니다,님 맛있게 드시고 맛난 사진과 소중한 리뷰 감사드립니다. 더 맛있게 더 정성으로 ...,5,1,0.517685
8678,비가와서 배달은 살짝 늦었지만 서비스로 튀김도 주시고 너무 만족스럽습니다~ 다음에 ...,아구 궂은 날씨엔 아무래도 배달이 조금 밀리곤 한답니다! 기사님들의 안전을 위해서 ...,5,1,0.752327


In [ ]:
train = pd.concat([train_neg, train_pos], ignore_index=True)
train['sentiment'].value_counts().sort_index()

0    8680
1    8680
Name: sentiment, dtype: int64

In [ ]:
#다시 섞기
train = train.sample(frac=1, random_state=42).reset_index(drop=True)
train

,review,reply,rating,sentiment,score
0,초밥 횟감이 신선하지 않았습니다. 저녁 챙겨드시라고 선물해 드린 분께도 죄송한 마음...,아고 속상해라. 먼저 아쉬움 드리게 되어 너무 안타깝고 죄송한 마음입니다 오래된 재...,1,0,0.920956
1,맛이 없어서 실망했어요.,"안녕하세요 고객님, 맛에 대한 실망을 느끼셨다니 저희도 매우 안타깝습니다. 좀 더...",5,0,0.919773
2,"양도 많고, 특히 곱창이 많이 있었어요. 납짝당면은 사랑이죠. 맛있습니다",안녕하세요.삽교원조두리곱창입니다. 저희 음식이 입맛에 맞으셨다니 정말 다행이고 뿌듯...,5,1,0.564190
3,너무 싱겁고 맛이 없어서 좀 실망했어요.,싱겁고 맛이 없어서 실망하셨다니 정말 죄송합니다. 고객님의 의견을 고려하여 개선해...,5,0,0.921372
4,배달 빠르고 맛도 좋아요! 다음에 또 시켜 먹을께요! 잘 먹었습니다,안녕하세여! 푸짐하구 맛있는 헬로팬입니다 맛있게 잘 드셔주신것 같아 기분 좋네요 ...,5,1,0.704180
...,...,...,...,...,...
17355,양도많고 배달도 빠르고 맛나게 먹었어요!,이용해 주셔 감사 드립니다 항상 건강하시고 행복하세요 감사합니다,5,1,0.685482
17356,맛있게 잘먹었어용 허니감자도 꿀맛,당신은 지금 치킨이 땡긴다 헬리오시티점 입니다 이용해 주셔서 감사드리구 맛있게 드...,5,1,0.512920
17357,식후감 힘내세요,주문해 주셔서 감사합니다. 늘 한결같은 마음과 맛으로 정성을 다 하겠습니다. 맛있는...,5,0,0.395288
17358,"피자의 토핑이 신선하지 않고, 피자도 맛이 별로였네요. 다음에는 다른 가게에서 시...",토핑이 신선하지 않고 맛도 좋지 않았다니 정말 죄송합니다. 앞으로는 고객님의 기대...,5,0,0.913409


# 데이터 준비

## 단어 사전 만들기

In [ ]:
review = train['review']
reply = train['reply']

In [ ]:
PAD = 0
SOS = 1
EOS = 2

class WordVocab():
    def __init__(self):
        self.word2index = {'<PAD>': PAD,
                           '<SOS>': SOS,
                           '<EOS>': EOS,}
        self.word2count = {}
        self.index2word = {PAD : '<PAD>',
                           SOS : '<SOS>',
                           EOS : '<EOS>'}
        self.n_words = 3  # PAD, SOS, EOS 포함

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
print(f'원문: {review[0]}')
lang = WordVocab()
lang.add_sentence(review[0])
print('단어사전: ', lang.word2index)

원문: 초밥 횟감이 신선하지 않았습니다. 저녁 챙겨드시라고 선물해 드린 분께도 죄송한 마음입니다. 
단어사전:  {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '초밥': 3, '횟감이': 4, '신선하지': 5, '않았습니다.': 6, '저녁': 7, '챙겨드시라고': 8, '선물해': 9, '드린': 10, '분께도': 11, '죄송한': 12, '마음입니다.': 13, '': 14}


## 데이터셋 만들기

In [ ]:
from konlpy.tag import Okt

In [ ]:
class TextDataset(Dataset):
    def __init__(self, df, min_length=3, max_length=32):
        super().__init__()

        # TOKEN 정의
        self.PAD = 0
        self.SOS = 1
        self.EOS = 2

        self.tagger = Okt() # 트위터 기반 형태소 분석기
        self.max_length = max_length

        # 형태소 처리와 단어사전 정의가 된 토큰을 담은 리스트
        src_clean = []
        tgt_clean = []

        # 단어 사전 생성
        wordvocab = WordVocab()

        for _, row in df.iterrows():
            # df 에서 'src'와 'tgt' 정의
            src = row['review']
            tgt = row['reply']

            # 형태소 처리
            src = self.clean_text(src)
            tgt = self.clean_text(tgt)

            if len(src.split()) > min_length and len(tgt.split()) > min_length:
                # 최소 길이를 넘어가는 문장의 단어만 추가
                wordvocab.add_sentence(src)
                wordvocab.add_sentence(tgt)
                src_clean.append(src)
                tgt_clean.append(tgt)

        self.srcs = src_clean
        self.tgts = tgt_clean
        self.wordvocab = wordvocab

    def clean_text(self, sentence):
        # 형태소 처리
        sentence = self.tagger.morphs(sentence)
        sentence = ' '.join(sentence)
        sentence = sentence.lower()
        return sentence

    def texts_to_sequences(self, sentence):
        # 문장 -> 시퀀스로 변환
        return [self.wordvocab.word2index[w] for w in sentence.split()]

    def pad_sequence(self, sentence_tokens):
        # 문장의 맨 끝 토큰은 제거
        sentence_tokens = sentence_tokens[:(self.max_length-1)]
        token_length = len(sentence_tokens)

        # 문장의 맨 끝부분에 <EOS> 토큰 추가
        sentence_tokens.append(self.EOS)

        for i in range(token_length, (self.max_length-1)):
            # 나머지 빈 곳은 <PAD> 토큰으로 채우기
            sentence_tokens.append(self.PAD)
        return sentence_tokens

    def __len__(self):
        return len(self.srcs)

    def __getitem__(self, idx):
        inputs = self.srcs[idx]
        inputs_sequences = self.texts_to_sequences(inputs) # 시퀀스로 바꾸고
        inputs_padded = self.pad_sequence(inputs_sequences) # pad 토큰 채우기

        outputs = self.tgts[idx]
        outputs_sequences = self.texts_to_sequences(outputs)
        outputs_padded = self.pad_sequence(outputs_sequences)

        return torch.tensor(inputs_padded), torch.tensor(outputs_padded)

In [ ]:
MAX_LENGTH = 32
dataset = TextDataset(train, min_length=3, max_length=MAX_LENGTH)

## 테스트 데이터 split

In [ ]:
x, y = dataset[0]

In [ ]:
print(f'x shape: {x.shape}')
print(x)

x shape: torch.Size([32])
tensor([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,  8,  2,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])


In [ ]:
print(f'y shape: {y.shape}')
print(y)

y shape: torch.Size([32])
tensor([19, 20, 21,  8, 22, 23, 24, 25, 26, 27, 16, 17, 18, 28, 29, 30, 31, 32,
        33, 34, 35, 36, 37, 38, 39, 40, 29, 41, 42, 43, 44,  2])


In [ ]:
from torch.utils.data import SubsetRandomSampler, random_split

train_size = int(len(dataset) * 0.8) # 80%의 데이터를 train에 할당
test_size = len(dataset) - train_size # 나머지 20% 데이터를 test에 할당
print(train_size, test_size)
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

13076 3270


## 데이터로더 만들기

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [ ]:
batch = next(iter(train_loader))
batch

[tensor([[  475,   302,    77,  5841,   295,    43,    77,  3060,  3623,  1132,
           1227,  3129,  3060,    83,  1060,    76,  1707,    83,   238,   823,
            411,    89,     5,   918, 18967, 18968,     8,  1237, 12685,  1494,
           1471,     2],
         [   70,   107,  2586,  3861,  3953,  1451,   186,  6087,     2,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0],
         [  130,    35,   168,   243,     2,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0],
         [  156,    60,    89,    77,  1275,  4788,   192,   193,     8,   165,
            198,    87,    65,   946,   595,     8,     2,     0,     0,     0,
              0,     0,     0,     0,     0, 

In [ ]:
# 1개의 배치 데이터를 추출
x, y = next(iter(train_loader))

# shape: (batch_size, sequence_length)
x.shape, y.shape

(torch.Size([16, 32]), torch.Size([16, 32]))

# Seq2Seq 모델링 (GRU)

## Encoder 만들기

In [ ]:
class Encoder(nn.Module):
    def __init__(self, num_embeddings, hidden_size, embedding_dim, num_layers):
        super().__init__()
        self.num_embeddings = num_embeddings # vocab size

        # 임베딩 레이어 정의 (number of vocabs, embedding dimension)
        self.emb_layer = nn.Embedding(num_embeddings, embedding_dim)

        # GRU (embedding dimension)
        self.rnn_layer = nn.GRU(embedding_dim,
                          hidden_size,
                          num_layers=num_layers,
                          bidirectional=False)

    def forward(self, x):
        x = self.emb_layer(x).permute(1, 0, 2)
        outputs, hn = self.rnn_layer(x)
        return outputs, hn

## Decoder 만들기

In [ ]:
class Decoder(nn.Module):
    def __init__(self, num_embeddings, hidden_size, embedding_dim, num_layers=1, dropout=0.2):
        super().__init__()
        self.num_embeddings = num_embeddings
        self.emb_layer = nn.Embedding(num_embeddings, embedding_dim)
        self.dropout = nn.Dropout(dropout)
        self.rnn_layer = nn.GRU(embedding_dim,
                          hidden_size,
                          num_layers=num_layers,
                          batch_first=False) # (1, batch_size) 로 변환을 쉽게 하기 위함

        # 최종 출력은 단어사전의 개수(num_embeddings)
        self.fc_layer = nn.Linear(hidden_size, num_embeddings)

    def forward(self, x, hidden_state):
        x = x.unsqueeze(0) # (1, batch_size) 로 변환
        x = F.relu(self.emb_layer(x))
        x = self.dropout(x)
        outputs, hn = self.rnn_layer(x, hidden_state)
        outputs = self.fc_layer(outputs.squeeze(0)) # (sequence_length, batch_size, hidden_size(32))
        return outputs, hn

## Seq2Seq 모델 만들기

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, inputs, outputs, teacher_forcing_ratio=0.5):
        # inputs : (batch_size, sequence_length)
        # outputs: (batch_size, sequence_length)

        batch_size, output_length = outputs.shape
        output_num_vocabs = self.decoder.num_embeddings

        # 리턴할 예측된 outputs를 저장할 임시 변수
        # (sequence_length, batch_size, num_vocabs)
        predicted_outputs = torch.zeros(output_length, batch_size, output_num_vocabs).to(self.device)

        # 인코더에 입력 데이터 주입, encoder_output은 버리고 hidden_state 만 살림(Attention 안하니까 필요없음)
        # 여기서 hidden_state가 디코더에 주입할 context vector 가 됨
        # (number of layers(1), batch_size, hidden_size)
        _, decoder_hn = self.encoder(inputs)

        # (batch_size) shape의 SOS TOKEN으로 채워진 디코더 입력 생성
        decoder_input = torch.full((batch_size,), SOS, device=self.device)

        # 반복문으로 출력 단어를 생성
        # 0번째는 SOS TOKEN이 위치하므로, 1번째 인덱스부터 순회
        for t in range(0, output_length):
            # decoder_input : 디코더 입력 (batch_size) 형태의 SOS TOKEN로 채워진 입력
            # decoder_output: (batch_size, num_vocabs)
            # decoder_hidden: (number of layers(1), batch_size, hidden_size), context vector와 동일 shape
            decoder_output, decoder_hn = self.decoder(decoder_input, decoder_hn)

            # t번째 단어에 디코더의 output 저장
            predicted_outputs[t] = decoder_output

            # teacher forcing 적용 여부 확률로 결정
            # 정답치를 다음 RNN Cell의 입력으로 넣어주는 경우. 수렴속도가 빠를 수 있으나, 불안정할 수 있음
            teacher_force = random.random() < teacher_forcing_ratio

            # top1 단어 토큰 예측
            top1 = decoder_output.argmax(1)

            # teacher forcing 인 경우 ground truth 값을, 그렇지 않은 경우, 예측 값을 다음 input으로 지정
            decoder_input = outputs[:, t] if teacher_force else top1

        return predicted_outputs.permute(1, 0, 2) # (batch_size, sequence_length, num_vocabs)로 변경

In [ ]:
# Encoder 정의
encoder = Encoder(num_embeddings=dataset.wordvocab.n_words,
                       hidden_size=32,
                       embedding_dim=64,
                       num_layers=1)
# Decoder 정의
decoder = Decoder(num_embeddings=dataset.wordvocab.n_words,
                       hidden_size=32,
                       embedding_dim=64,
                       num_layers=1)
# Seq2Seq 정의
seq2seq = Seq2Seq(encoder, decoder, 'cpu')

In [ ]:
x, y = next(iter(train_loader))
print(x.shape, y.shape)
# (batch_size, sequence_length), (batch_size, sequence_length)

torch.Size([16, 32]) torch.Size([16, 32])


In [ ]:
output = seq2seq(x, y)
print(output.shape)
# (batch_size, sequence_length, num_vocabs)

torch.Size([16, 32, 27306])


## 모델 확인

In [ ]:
NUM_VOCABS = dataset.wordvocab.n_words
HIDDEN_SIZE = 512
EMBEDDIMG_DIM = 256

print(f'num_vocabs: {NUM_VOCABS}\n======================')

# Encoder 정의
encoder = Encoder(num_embeddings=NUM_VOCABS,
                  hidden_size=HIDDEN_SIZE,
                  embedding_dim=EMBEDDIMG_DIM,
                  num_layers=1)
# Decoder 정의
decoder = Decoder(num_embeddings=NUM_VOCABS,
                  hidden_size=HIDDEN_SIZE,
                  embedding_dim=EMBEDDIMG_DIM,
                  num_layers=1)

# Seq2Seq 생성
# encoder, decoder를 device 모두 지정
model = Seq2Seq(encoder.to(device), decoder.to(device), device)
print(model)

num_vocabs: 27306
Seq2Seq(
  (encoder): Encoder(
    (emb_layer): Embedding(27306, 256)
    (rnn_layer): GRU(256, 512)
  )
  (decoder): Decoder(
    (emb_layer): Embedding(27306, 256)
    (dropout): Dropout(p=0.2, inplace=False)
    (rnn_layer): GRU(256, 512)
    (fc_layer): Linear(in_features=512, out_features=27306, bias=True)
  )
)


# 모델 학습

## 하이퍼파라미터

In [ ]:
class EarlyStopping:
    def __init__(self, patience=3, delta=0.0, mode='min', verbose=True):
        """
        patience (int): loss or score가 개선된 후 기다리는 기간. default: 3
        delta  (float): 개선시 인정되는 최소 변화 수치. default: 0.0
        mode     (str): 개선시 최소/최대값 기준 선정('min' or 'max'). default: 'min'.
        verbose (bool): 메시지 출력. default: True
        """
        self.early_stop = False
        self.patience = patience
        self.verbose = verbose
        self.counter = 0

        self.best_score = np.Inf if mode == 'min' else 0
        self.mode = mode
        self.delta = delta


    def __call__(self, score):

        if self.best_score is None:
            self.best_score = score
            self.counter = 0
        elif self.mode == 'min':
            if score < (self.best_score - self.delta):
                self.counter = 0
                self.best_score = score
                if self.verbose:
                    print(f'[EarlyStopping] (Update) Best Score: {self.best_score:.5f}')
            else:
                self.counter += 1
                if self.verbose:
                    print(f'[EarlyStopping] (Patience) {self.counter}/{self.patience}, ' \
                          f'Best: {self.best_score:.5f}' \
                          f', Current: {score:.5f}, Delta: {np.abs(self.best_score - score):.5f}')

        elif self.mode == 'max':
            if score > (self.best_score + self.delta):
                self.counter = 0
                self.best_score = score
                if self.verbose:
                    print(f'[EarlyStopping] (Update) Best Score: {self.best_score:.5f}')
            else:
                self.counter += 1
                if self.verbose:
                    print(f'[EarlyStopping] (Patience) {self.counter}/{self.patience}, ' \
                          f'Best: {self.best_score:.5f}' \
                          f', Current: {score:.5f}, Delta: {np.abs(self.best_score - score):.5f}')


        if self.counter >= self.patience:
            if self.verbose:
                print(f'[EarlyStop Triggered] Best Score: {self.best_score:.5f}')
            # Early Stop
            self.early_stop = True
        else:
            # Continue
            self.early_stop = False

In [ ]:
LR = 1e-3
optimizer = optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.CrossEntropyLoss()
es = EarlyStopping(patience=5, delta=0.001, mode='min', verbose=True)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

## 학습 loop 만들기

In [ ]:
def train(model, data_loader, optimizer, loss_fn, device):
    model.train()
    running_loss = 0

    for x, y in data_loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()

        # output: (batch_size, sequence_length, num_vocabs)
        output = model(x, y)
        output_dim = output.size(2)

        # 1번 index 부터 슬라이싱한 이유는 0번 index가 SOS TOKEN 이기 때문
        # (batch_size*sequence_length, num_vocabs) 로 변경
        output = output.reshape(-1, output_dim)

        # (batch_size*sequence_length) 로 변경
        y = y.view(-1)

        # Loss 계산
        loss = loss_fn(output, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * x.size(0)

    return running_loss / len(data_loader)

In [ ]:
def evaluate(model, data_loader, loss_fn, device):
    model.eval()

    eval_loss = 0

    with torch.no_grad():
        for x, y in data_loader:
            x, y = x.to(device), y.to(device)
            output = model(x, y)
            output_dim = output.size(2)
            output = output.reshape(-1, output_dim)
            y = y.view(-1)

            # Loss 계산
            loss = loss_fn(output, y)

            eval_loss += loss.item() * x.size(0)

    return eval_loss / len(data_loader)

## 추론 함수

In [ ]:
def sequence_to_sentence(sequences, index2word):
    outputs = []
    for p in sequences:

        word = index2word[p]
        if p not in [SOS, EOS, PAD]:
            outputs.append(word)
        if word == EOS:
            break
    return ' '.join(outputs)

## 테스트 loop 만들기
- 챗봇 함수화

In [ ]:
# 전체 예측값을 데이터프레임 형식에 담기
def random_evaluation(model, dataset, index2word, device, n=10):

    n_samples = len(dataset)
    indices = list(range(n_samples))
    np.random.shuffle(indices)      # Shuffle
    sampled_indices = indices[:n]   # Sampling N indices

    # 샘플링한 데이터를 기반으로 DataLoader 생성
    sampler = SubsetRandomSampler(sampled_indices)
    sampled_dataloader = DataLoader(dataset, batch_size=10, sampler=sampler)

    model.eval()
    with torch.no_grad():
        for x, y in sampled_dataloader:
            x, y = x.to(device), y.to(device)
            output = model(x, y, teacher_forcing_ratio=0)
            # output: (number of samples, sequence_length, num_vocabs)

            preds = output.detach().cpu().numpy()
            x = x.detach().cpu().numpy()
            y = y.detach().cpu().numpy()

            data = {'test_review': [], 'chatbot_reply': []}
            for i in range(n_samples):
                x_result = sequence_to_sentence(x[i], index2word)
                y_result = sequence_to_sentence(y[i], index2word)
                pred_result = sequence_to_sentence(preds[i].argmax(1), index2word)
                pred_result_correct = correct_spell(pred_result)
                data['test_review'].append(x_result)
                data['chatbot_reply'].append(pred_result_correct)

    df = pd.DataFrame(data)
    print(df)

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

def calculate_bleu(text, generated_text):
    y = [text]  # 실제 데이터 y
    X = generated_text  # 생성된 텍스트 X

    # BLEU 점수 계산
    bleu_score = sentence_bleu(y, X)
    return bleu_score

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.2 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline
pipe = pipeline("text2text-generation", model="lcw99/t5-base-korean-paraphrase")

def paraphrase_text(text):
    generated_text = pipe(text, max_length=128, early_stopping=True,
                          num_beams=4,
                          num_return_sequences=2,
                          no_repeat_ngram_size=4,
                          top_k=5,
                          top_p=0.95)
    generated_text = generated_text[0]['generated_text']
    return generated_text

In [ ]:
# 10개만 보기
def random_evaluation_10(model, dataset, index2word, device, n=10):

    n_samples = len(dataset)
    indices = list(range(n_samples))
    np.random.shuffle(indices)      # Shuffle
    sampled_indices = indices[:n]   # Sampling N indices

    # 샘플링한 데이터를 기반으로 DataLoader 생성
    sampler = SubsetRandomSampler(sampled_indices)
    sampled_dataloader = DataLoader(dataset, batch_size=10, sampler=sampler)

    model.eval()
    with torch.no_grad():
        for x, y in sampled_dataloader:
            x, y = x.to(device), y.to(device)
            output = model(x, y, teacher_forcing_ratio=0)
            # output: (number of samples, sequence_length, num_vocabs)

            preds = output.detach().cpu().numpy()
            x = x.detach().cpu().numpy()
            y = y.detach().cpu().numpy()

            for i in range(n):
                result_review = sequence_to_sentence(x[i], index2word)
                result_reply = sequence_to_sentence(y[i], index2word)
                result_pred = sequence_to_sentence(preds[i].argmax(1), index2word)
                print(f'리뷰    : {result_review}')
                print(f'답변    : {result_reply}')
                print(f'예측답변(paraphrase): {paraphrase_text(result_pred)}') # paraphrased text 반환
                print(f'예측답변 : {result_pred}')
                print(f'BLEU점수: {calculate_bleu(result_reply, result_pred)}')
                print('==='*10)

## 학습하기

In [ ]:
# yogiyo_paraphrase
NUM_EPOCHS = 10
STATEDICT_PATH = f'{DATA_PATH}gru_model.pt'

best_loss = np.inf

for epoch in tqdm(range(NUM_EPOCHS), total=NUM_EPOCHS):
    loss = train(model, train_loader, optimizer, loss_fn, device)

    val_loss = evaluate(model, test_loader, loss_fn, device)

    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), STATEDICT_PATH)

    if epoch % 5 == 0:
        print(f'epoch: {epoch+1}, loss: {loss:.4f}, val_loss: {val_loss:.4f}')

    # Early Stop
    es(loss)
    if es.early_stop:
        break

    # Scheduler
    scheduler.step(val_loss)

model.load_state_dict(torch.load(STATEDICT_PATH))
torch.save(model.state_dict(), f'{DATA_PATH}gru_model_0914{best_loss:.4f}.pt')

  0%|          | 0/10 [00:00<?, ?it/s]

epoch: 1, loss: 58.7606, val_loss: 62.0905
[EarlyStopping] (Update) Best Score: 58.76057
[EarlyStopping] (Update) Best Score: 54.55514
[EarlyStopping] (Update) Best Score: 51.03490
[EarlyStopping] (Update) Best Score: 47.43922
[EarlyStopping] (Update) Best Score: 44.29853
Epoch 00005: reducing learning rate of group 0 to 5.0000e-04.
epoch: 6, loss: 40.1045, val_loss: 63.6829
[EarlyStopping] (Update) Best Score: 40.10448
[EarlyStopping] (Update) Best Score: 37.76087
[EarlyStopping] (Update) Best Score: 35.88744
Epoch 00008: reducing learning rate of group 0 to 2.5000e-04.
[EarlyStopping] (Update) Best Score: 33.60856
[EarlyStopping] (Update) Best Score: 32.27231


# 추론 및 답변생성

In [ ]:
STATEDICT_PATH = f'{DATA_PATH}gru_model_0914{best_loss:.4f}.pt'
model.load_state_dict(torch.load(STATEDICT_PATH))
random_evaluation_10(model, test_dataset, dataset.wordvocab.index2word, device)

리뷰    : 짜장 소스 좀 짜고 양 이 아쉽내요 . 만두 살짝 냄새 나 요 . 하나 먹고 버렸어요
답변    : 먼저 만족 시켜 드리지 못 해 정말 죄송합니다 . 그럼에도 불구 하고 이렇게 리뷰 남겨주셔서 너무 감사합니다 ! 맛있는 음식 으로 찾아가기 위해 신경 쓰며 노력 중 이지만 아직 미흡한 부분


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


예측답변(paraphrase): 저희 식당 조 두리 곱창 입니다 . 고객 님이 이렇게 좋은 음식을 대접해 주셔서 감사합니다 .
예측답변 : 안녕하세요 . 삽 교원 조 두리 곱창 입니다 . 저희 음식 이 입맛 에 맞으셨다니 정말 다행 이고 뿌듯합니다 코로나 로 인해 많이 힘든시기에 고객 님 덕분 에 큰
BLEU점수: 0.13703486199568538
리뷰    : 배달 이 늦는데 음식 도 별로 였어요 . 좋은 경험 이 아니었어요
답변    : 음식 이 지연 되었고 , 맛 도 별로 였다는 점 깊이 양해 의 말씀 을 드립니다 . 저희 식당 은 항상 신속하고 정확한 배달 을 위해 최선 을 다 하고 있습니다
예측답변(paraphrase): 더 좋은 서비스를 제공 할 수 있도록 노력 하겠습니다. 배달 시간이 늦어서 음식이 식어버린 점 사과 드립니다.
예측답변 : 배달 이 늦어서 음식 이 식어 버린 점 에 대해 죄송합니다 . 더 좋은 서비스 를 제공 할 수 있도록 노력 하겠습니다 .
BLEU점수: 0.14589628638904711
리뷰    : 리뷰 이벤 참여 한다고 했는데 서비스 안 주셨 네 욤 상점 찜 은 어디서 하나요 하는 곳 이 없던데 그래도 맛있었으니 별 5 개 드려요 ~ 독박 육아 중 이라 이
답변    : 어라 아이디 를 적으셨는데도 서비스 만두 가 없나요 ? 죄송합니다 ~ 독박 육아 힘내시고 버티고 즐기시다 보면 시간 이 지나고 원하지 않아도 아이 는 크게 되어있습니다 . 화이팅 ~!
예측답변(paraphrase): 고객 님 ! 주문해 주셔서 감사합니다 아주 빠른 배송 덕분에 별 5점 드리고, 고객님이 맛있게 먹어 줘서 또 감사드립니다.
예측답변 : 안녕하세요 ! 고객 님 ! 주문 해주셔서 감사합니다 바쁘신 와중 에 별 점 5 점 ! 리뷰 감사 드립 다 , 고객 님 께서 맛있게 드신것 같아 너무 너무 다행 스럽
BLEU점수: 0.04904704449077316
리뷰    : 양도 푸 짐 하고 맛 도 너무 좋네요 시원하게 주신 커피 도 너무 잘 먹었습니다 

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


예측답변(paraphrase): 저희는 항상 고객 님들께 맛있는 요리를 제공 하기 위해 노력 하고 있습니다.
예측답변 : 맛 과 양 에 대해 실망 을 드려 죄송합니다 . . 저희 는 항상 고객 님 들께 맛있는 음식 을 제공 하기 위해 노력 하고 있습니다 . 다음 에는
BLEU점수: 0.44326143331752793
리뷰    : 리뷰 식혜 가 없어도 맛있긴 하지만 아쉬워요 . 이번 에는 좀 실망 이었어요
답변    : 리뷰 식혜 가 없어서 아쉬우셨다니 죄송합니다 . 더 나은 서비스 를 위해 항상 노력 하고 있으니 다음 에는 더 만족스러운 경험 을 선사 해드리도록 하겠습니다 .
예측답변(paraphrase): 저희가 좀 더 많이 넣어 드려서 죄송합니다. 다음 엔 더 좋은 맛으로 찾아 뵙겠습니다.
예측답변 : 안녕하세요 . 저희 가 좀 더 많이 들어있지 . 죄송합니다 . 다음 에는 더 좋은 맛 을 제공 할 수 있도록 노력 하겠습니다 .
BLEU점수: 0.3479093384346797
리뷰    : 맛있게 잘 먹었습니다 김치전 도 잘 먹었습니다
답변    : 감사합니다 맛있게 드셔주 셧다 니 마음 가지 편안합니다 언제나 맛 으로 보답 드리는 수락 되겠습니다 !
예측답변(paraphrase): 항상 고객 여러분의 행복한 식사 자리를 위해 노력하는 청춘 식당이 되도록 노력하겠습니다.
예측답변 : 맛있게 드셔주셔서 감사 드립니다 ~ 항상 고객 님 의 행복한 식사 를 위해 최선 을 다 하는 청춘 식당 이 되겠습니다 ~! 언제 든지 김치 찜 생각나시면 청춘 식당
BLEU점수: 0.18589999836775795
리뷰    : 날 이 너무 더워서 그런지 생크림 이 좀 흐른 채 로 배송 되었지만 맛 났습니다
답변    : 오늘 저희 매장 에 발걸음 을 해주셔서 영광 입니다 메뉴 시키실 때 마다 항상 많이 고민 하셨을 텐데 ! 기대하시는 맛 , 원하시는 맛 드릴 수 있도록 노력 할게요 ! 다음
예측답변(paraphrase): 다음 에는 더 맛있는 요리로 찾아 뵐게요 . 감사합니

In [ ]:
# # 0905 데이터 + over + 긍부정 비율 2:1 + 에폭 30 + paraphrase
# STATEDICT_PATH = '/content/drive/MyDrive/project_final/data/models/yogiyo_0905_over.pt'
# model.load_state_dict(torch.load(STATEDICT_PATH))
# random_evaluation_10(model, test_dataset, dataset.wordvocab.index2word, device)

리뷰    : 배달 음식 자주 시켜 먹고 여기 서는 처음 인데 1시간 10분 만에 배달 오고 늦게 온 것 도 그런데 떡 다 불어있고 치즈 뭉쳐서 딱딱하구요 실망 배달 시켜서 기분 안좋긴 처음
답변    : 지금 리뷰 를 봤네요 언제 주문 하신거죠 저희 가 늦게 간다 고 해서 음식 도 오래되는건 극히 드문 일이 에요 늦게가는건 주문 이 밀려서 늦게 만들었기 때문 인데 요 원래 좀
예측답변: 저희 가게를 방문해 주신 고객님께 감사의 뜻을 전하고 싶고 앞으로도 많은 이용 부탁 드리겠습니다.
BLEU점수: 0.09267714327273047
리뷰    : 배달 도 적당한 시간 에 왔는 댕 수 저 젓가락 이 안 왔어요 o 표시 햇 는대 사무실 에서 주 문 한 거 라 너무 당황 했 네 여 담 엔
답변    : 아이고 소중한 고객 님 사죄 의 말씀드립니다 본의 아니게 인도 식 의 식사 로 대접 해드려 버렸네요 다음 엔 절대 절대 네버 빼먹지 않고 잘 챙겨 드리겠습니다 넘 넘 죄송합니다 혹시
예측답변: 저희 음식 은 배달 대행 업체 를 이용 해 요리 과정 에서 부족했던 을 하고 있습니다 일부 용기 에 따라 일부 과정 에 미흡했던 을하고 있습니다
BLEU점수: 0.09100168359620395
리뷰    : 잘 먹었고 맛있게 잘 먹었어요
답변    : 즐거운 여름 휴가 를 떠나시기 전 에 맛있는 음식 으로 건강 도 챙겨 보세요 .
예측답변: 감사합니다. 감사합니다.
BLEU점수: 8.016714023843699e-156
리뷰    : 넘 맛있고 좋았는데 치즈 는 추가 가 된 건지 의문 이 살짝 드네 요
답변    : 맛있게 드셨다니 감사합니다 더 푸 짐하도록 신경 쓰겠습니다 항상 노력 하는 되겠습니다 즐거운 시간 보내시고 행복한 나날 되세요
예측답변: 저희 매장은 더욱 좋은 요리와 서비스로 찾아 뵙겠습니다 고객님 감사합니다.
BLEU점수: 0.09871860383734853
리뷰    : 잘 먹었 습 니당
답변    : 너무 다행 저희 가 준비 한 음식 맛있게 드셔주셔서요 

In [ ]:
# 0905 데이터 + 긍부정 비율 2:1 + 에폭 30 + paraphrase
STATEDICT_PATH = '/content/drive/MyDrive/project_final/data/models/yogiyo_0905.pt'
model.load_state_dict(torch.load(STATEDICT_PATH))
random_evaluation_10(model, test_dataset, dataset.wordvocab.index2word, device)

리뷰    : 갈비 진짜 너무 맛있네용 반신반의 하면서 시 켯 는데 갈비 맛 나구 진짜 맛 잇 어유 치킨 도 구 웃 잘 먹었 슴 당
답변    : 안녕하세요 맛있게 드셔주셔서 감사합니다 치킨 조합 정말 별미 고객 님 께서 치킨 이 땡기실 때 마다 저희 가 생각나도록 더욱 열심히 하겠습니다 아 주문 정말 감사 드리고 다음 에 또 봐요


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


예측답변: 저희 음식이 입맛에 맞았다면 지금처럼 힘든 시기는 없었을 것이라 생각하는데 고객 여러분의 소중한 후기 덕분입니다 감사합니다.
BLEU점수: 0.2696986267476108
리뷰    : 맛있는 냄새 에 잘 먹을게요 비 조리 인데 아쉽게 리뷰 이벤트 안 적어서 인지 사리 가 없네요
답변    : 에 이 쿵 아쉬워서 어 쩌지 요 말 씀이 없으시면 챙겨 드리 기 가 힘듭니다 다음 에는 꼭 적어 주세요 더 맛있게 드실 수 있게 같이 드릴게요
예측답변: 저희가 이 부분은 운영을 이쪽에서 하기 때문에 고객님들이 불편을 많이 느끼셨겠습니다만 양해 부탁드립니다.
BLEU점수: 3.176496675533262e-78
리뷰    : 옆집 문 앞 에 배달 해주셨네요 맛있어서 다 먹고 사짘 찍어 버렸어요 잘 먹었습니다
답변    : 앗 죄송합니다 고객 님 그래도 너그러이 이해 해주시고 리뷰 까지 남겨주셔서 너무 감사해요 더 노력 하는 되겠습니다 늘 행복하세용


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


예측답변: 안녕하세요 고객 님 다음에도 더 좋은 서비스와 맛있는 요리로 찾아 뵙겠습니다.
BLEU점수: 0.24830724520513484
리뷰    : 몸 이 안좋아서 시켜 봤는데 양도 많고 맛 도 좋아요
답변    : 리뷰 남겨주셔서 감사합니다 칭찬 해 주시 니 힘 이 팍팍 나네요 저희 음식 이 기력 을 회복하시는데 도움 이 되기만 한다면 그것 만으로도 너무나 행복합니다 발전 하는 맛 과 좋은 서비스
예측답변: 고객 여러분 감사합니다 이렇게 좋은 요리 만들어 주셔서 요즘 같이 힘든 시기에 정말 감사드립니다.
BLEU점수: 0.22491637465159936
리뷰    : 먹고나면 속이 편하고 확실히 맛있어요 가라 아케 도 오랜 만 에 시켜 먹었는데 잘 먹었습니다
답변    : 반갑습니다 오늘 도 맛있게 드시고 예쁜 사진 리뷰 정말 감사 드립니다
예측답변: 고객 님 맛있게 먹어주셔서 감사합니다.
BLEU점수: 0.19154624426653655
리뷰    : 맛 도 좋고 배달 도 빨 라서 따뜻하게 맛있게 먹었어요
답변    : 즐거운 시간 보내시고 소중한 리뷰 작성 해주셔서 감사 드려요 메뉴 별로 가장 맛있는 온도 가 있다고 해 요 많은 분 들 이 매장 에서 찾아주시는 이유 이기도 하죠 저희 도 이러한
예측답변: 당일 삶은 보쌈 직접 담근 양념과 신선한 음식 재료로 고객 밥상에 항상 오르는 것이 바로 이 요리입니다 맛있게 먹어 주셔서 감사합니다
BLEU점수: 0.07196816797546395
리뷰    : 폴트 버거 너무 맛있고 감자 튀김 도 정말 맛있네요 배달 도 빨리 해주셔서 따뜩하 게 잘 먹었어요 포장 도 너무 예쁘고 테니스 컨셉 매장 도 꼭 한번 가보고싶어요
답변    : 소중한 리뷰 글 감사 드립니다 저희 항상 고객 님 을 향 한 마음 으로 맛있는 수제 찾아 뵙도록 노력 하겠습니다 감사합니다
예측답변: 당일 삶은 삼겹살과 직접 담근 김치, 그리고 신선한 식자재를 가지고 고객 밥상에 매번 정성을 다하는 요리 연구가 계속 되고 있습니다.
BLEU점수:

In [ ]:
# 0905 데이터 + 긍부정 비율 5:5 + 에폭 30
STATEDICT_PATH = '/content/drive/MyDrive/project_final/data/models/yogiyo_0905.pt'
model.load_state_dict(torch.load(STATEDICT_PATH))
random_evaluation_10(model, test_dataset, dataset.wordvocab.index2word, device)

리뷰    : 간장 은 묽어서 좀 아쉬운데 나머진 맛 잇어요
답변    : 고객 님 안녕하세요 골라 먹는 부천대 주문 에 맛있게 드셔 주셔서 정말 감사합니다 맛있게 드셨다는 칭찬 은 언제 들어도 참 기분 이 좋아지는 그런 말 인거 같아요 덕분 에 힘든 시기
예측답변: 고객 님 소중한 리뷰 감사합니다 저희는 저희 민의의 고기를 두 툼 하 게 썰고 있습니다 광어 도미 등 이 이 이 저희 매장으로 연락 주세요 묻고 따지지
BLEU점수: 0.1506279696450486
리뷰    : 오 여기 좋네요 시켜놓고 낮잠 자 다가 늦게 먹었긴 했는데 진짜 빨리 와요
답변    : 감사합니다 주문 즉시 최대한 빨리 조리 해서 보내 드리고있어요 빠른 베차 시 스팀 이기 에 고객 님 들이 우선 적 으로 잡아 가는 편이 에요
예측답변: 안녕하세요 고객 님 소중한 리뷰 감사합니다 주셔서 감사합니다 감사드립니다 항상 고객 님의 소리는 저희를 춤추게 합니다 앞으로 계속해서 정성스러운 음식을 제공해 드리도록 약속
BLEU점수: 0.18665030216111525
리뷰    : 닭발 좋아해서 여기저기 주문 해서 먹는데 공주 닭발 맛있습니다 배달 도 무지 빠르고 센스 있는 서비스 까지 감사합니다 맛있게 잘 먹었습니다
답변    : 항상 같은 맛 유지 하도록 노력 할게요 항상 맛 과 청결 에 정성 을 다 하겠습니다 주문 해주셔서 감사합니다
예측답변: 고객 님 소중한 리뷰 감사합니다 감사드립니다 고객 님의 만족을 위해 노력하는 되겠습니다 즐거운 시간 보내시고 행복한 나날 되세요
BLEU점수: 0.18289441873227932
리뷰    : 최고 너무 맛있고 배달 도 빠르고 잘 먹었습니다
답변    : 반갑습니다 맛있게 드셔주시고 잊지 않고 리뷰 남겨 주심 에 큰 힘 을 얻습니다 늘 변함 없는 맛 과 친절한 서비스 로 보답 하겠습니다 항상 행복하세요
예측답변: 안녕하세요 고객 님 저희 매장으로 찾아뵙겠습니다
BLEU점수: 0.0218228208440802
리뷰    : 오늘 은 주 문 한 거 빠

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


예측답변: 주문해 주셔서 감사합니다 드립니다 다음에 또 주문해 주세요
BLEU점수: 0.06122493138554343
리뷰    : 전체 적 으로 맛있네요 하와 이안 에 코코넛 은 좀 그렇긴 한데 요 기 서 만 계속 피자 먹네요
답변    : 맛있게 드셔주시고 소중한 리뷰 달아주셔서 정말 감사 드립니다
예측답변: 안녕하세요 고객 님 저희 매장을 찾아주셔서 너무 감사드립니다 앞으로도 저희 음식을 이용해 주셔서 정말 감사드립니다 다음에 또 주문해 주세요
BLEU점수: 0.18463832893957804
리뷰    : 사장 님 맛있게 잘 먹었습니다 번창 하세요
답변    : 안녕하세요 고객 님 맛있게 드시고 소중한 리뷰 와 별 5 개 감사합니다 언제 주문 하셔도 푸 짐 하고 맛있는 닭발 을 위해 항상 최선 을 다 하겠습니다 앞 으로도 많은 사랑
예측답변: 안녕하세요 고객 님 맛있게 드시고 소중한 리뷰 남겨주셔서 정말 감사합니다 드립니다 고객 님의 소리는 저희를 춤추게 합니다 앞으로 계속해서 정성스러운 음식을 제공해 드리도록 약속
BLEU점수: 0.4093854533293307


In [ ]:
# yogiyo 전체데이터 모델링
STATEDICT_PATH = '/content/drive/MyDrive/project_final/data/models/yogiyo_paraphrase.pt'
model.load_state_dict(torch.load(STATEDICT_PATH))
random_evaluation_10(model, test_dataset, dataset.wordvocab.index2word, device)

리뷰    : 먹을만 합니다 추천 해 요
답변    : 좋은 리뷰 감사 드립니다 저희 은 리뷰 평점 이나 표면 적 으로만 좋은 집 이 아닌 드셧 던 분들 이 믿고 다시 드시고싶은 집 이 되자 라는 마음 으로 모두 가
예측답변: 고객 님 소중한 고객 네 이 이임 이임 맛있게 드셨다니 너 어어 어 무 기쁜 네 용 만족스러운 식사하셨다면에 용 만족스러운 식사하셨다면 찜 가
BLEU점수: 1.7456357302038282e-78
리뷰    : 여친 이 너무 맛있게 잘 먹었답니다 감사합니다 사장 님 번창 하세요
답변    : 안녕하세요 토핑 을 아끼지 않는 미 미 주문 주시 고 리뷰 로 찾아와 주셔서 정말 감사 드립니다 앞 으로도 맛있는 생각나실 때 저희 미 또 찾아주세요 감사합니다
예측답변: 맛있게 드셔주셔서 감사합니다 요청 사항 은 조 리전 늘 꼼꼼하게 체크하고 있으니 필요하신 사항 있으시면 다음 주 문 시 주 문 메모에 남겨주세요 최대한 반영해서 보내 드리겠습니다 믿고 주문해
BLEU점수: 0.1556563403952443
리뷰    : 주말 의 마무리 는 역시
답변    : 고객 님 리뷰 에 몸 이 절로 들썩입니다 시간 날 때 마다 리뷰 훑어 보며 반응 살피 고 있는데 덕분 에 넘 힘 이나고 힐링 이 되네요 리뷰 너무 감사 드립니다
예측답변: 안녕하세요 고객 님 소중한 리뷰 남겨주셔서 감사합니다 저희 큰아들 백 순대는 항상 맛있는 서비스를 해드리기 위해 서 노력하고 있습니다 사용되는 모든 곱창 은 국내 산 제 주산
BLEU점수: 0.12288378213016357
리뷰    : 고기 가 맛있어요 사장 님 이 서비스 로 쌈장 도 주셨어요 배달 도 예상 시간 보다 빠흐 게 옵니다
답변    : 저희 목살 만족스럽고 맛있게 잘 드셨다니 기쁩니다 소중한 리뷰 와 저희 에게 큰 힘 을 주는 좋은 평가 남겨 주셔서 정말 감사 드립니다 무더운 날씨 에 건강 잘 지키시고 오늘 도
예측답변: 맛있게 드셔 주시 고 소중한 리뷰 남겨주셔서 감사합니다 만족스러운 식사하신 것 같아 오

In [ ]:
# yogiyo 전체데이터 모델링
STATEDICT_OV_PATH = '/content/drive/MyDrive/project_final/data/models/yogiyo_paraphrase_clean.pt'
model.load_state_dict(torch.load(STATEDICT_OV_PATH))
random_evaluation_10(model, test_dataset, train_dataset.wordvocab.index2word, device)

리뷰    : 주문 에 후라이드 잇 슈가 있었지만 다 맛있었습니다
답변    : 저희 을 항상 찾아주시고 성원 해주시는 분 들 께 감사 의 뜻 을 전하 고 싶을 만큼 귀한 시간 내어 주셔서 감사합니다 .
예측답변: 항상 저희 직구 삼이 직구 삼이 되도록 노력 직구 삼이 되도록 노력 하겠습니다 감사합니다 .
BLEU점수: 0.14828218247800706
리뷰    : 맛있습니다 배달 도 빠르고 양도 많아오
답변    : 항상 건강하고 행복한 나날 들 이 되시길 기원 합니다 . 고객 님 들의 소중한 정보 감사 드립니다 .
예측답변: 항상 건강하고 행복한 나날 들 이 되시길 기원 합니다 .
BLEU점수: 0.4464393987758162
리뷰    : 디저트 하나 하나 다 맛있어요 휘 낭시 에 랑 소금 빵 완전 맛있어요 이 즈 니 버터 프레첼 은 팥 앙금 들어있는건 줄 알았는데 다시 보니 아니더라 규 요 그치만 이
답변    : 이 즈 니 버터 프리첼 의 팥 앙금 버전 앙 버터 도 뚜레쥬르 팀 에 제안 해 보겠습니다 .
예측답변: 이 즈 니 버터 프리첼 의 팥 앙금 버전 앙 버터 도 뚜레쥬르 팀 에 제안 해 보겠습니다 .
BLEU점수: 1.0
리뷰    : 국물 이 깔끔하고 맛있어요 또 먹고 싶은 맛 이네 요
답변    : 저희 국밥 을 많이 사랑 해 주세요 :) 항상 최선 을 다해 준비 하는 만큼 더 좋은 맛 과 영양 으로 보답 하겠습니다 .
예측답변: 저희 국밥 을 많이 사랑 해 주세요 :) 항상 최선 을 다해 준비 하는 만큼 더 좋은 맛 과 영양 으로 보답 하겠습니다 .
BLEU점수: 1.0
리뷰    : 커피 도 부드럽고 다 맛 나네요 잘 먹었습니다
답변    : 항상 정갈한 브런치 맛있는 브런치 준비 하고 고객 님 기다릴게요 곧 또 만나요 그때 까지 행복한 시간 되세요 .
예측답변: 항상 정갈한 브런치 는 브런치 준비 가 준비 기다릴게요 기다릴게요 곧 또 찾아주세요 행복한 하루 되세요 .
BLEU점수: 0.5833316665807692
리뷰    : 맛